In [22]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.5
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.26
Branch HEAD
Compiled by user ubuntu on 2025-02-23T20:30:46Z
Revision 7c29c664cdc9321205a98a14858aaf8daaa19db2
Url https://github.com/apache/spark
Type --help for more information.


In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet


--2025-03-04 22:43:39--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.112, 3.164.82.197, 3.164.82.160, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  3.24MB/s    in 22s     

2025-03-04 22:44:01 (2.81 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [9]:
!ls

Untitled.ipynb                  env
entry.py                        yellow_tripdata_2024-10.parquet


SyntaxError: invalid syntax (292627728.py, line 1)

In [13]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [33]:
from pyspark.sql import types
schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("tpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("tpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("passenger_count", types.LongType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("RatecodeID", types.LongType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("payment_type", types.LongType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True),
    types.StructField("Airport_fee", types.DoubleType(), True),
])

In [34]:

df = spark.read.option("header", "true").schema(schema).parquet('yellow_tripdata_2024-10.parquet')

df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 03:30:44|  2024-10-01 03:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [38]:
df.repartition(4).write.parquet("./result")

In [51]:
import pyspark.sql.functions as f
df.filter(f.to_date(f.col('tpep_pickup_datetime')).like("%10-15")).count()

122561

In [46]:
df.registerTempTable('trips_data')

/opt/homebrew/Cellar/apache-spark/3.5.5/libexec/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [50]:
spark.sql (
    '''
    select count(*) from trips_data as t where month(t.tpep_pickup_datetime) == 10 and day(t.tpep_pickup_datetime) ==15;
    '''
).show()

+--------+
|count(1)|
+--------+
|  122561|
+--------+



In [76]:
spark.sql(
    """
    select tpep_dropoff_datetime - tpep_pickup_datetime as tm, extract(days from tm) as days, extract(hours from tm), extract(seconds from tm)  from trips_data
    order by tm desc
    """
).show()

+--------------------+----+---------------------------------------------+-----------------------------------------------+
|                  tm|days|extract(hours FROM lateralAliasReference(tm))|extract(seconds FROM lateralAliasReference(tm))|
+--------------------+----+---------------------------------------------+-----------------------------------------------+
|INTERVAL '6 18:37...|   6|                                           18|                                       4.000000|
|INTERVAL '5 23:19...|   5|                                           23|                                      30.000000|
|INTERVAL '5 17:45...|   5|                                           17|                                      38.000000|
|INTERVAL '4 18:50...|   4|                                           18|                                       5.000000|
|INTERVAL '3 17:53...|   3|                                           17|                                      54.000000|
|INTERVAL '3 17:26...|  

In [77]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-05 00:51:31--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.112, 3.164.82.40, 3.164.82.197, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-05 00:51:32 (980 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [78]:
!ls

Untitled.ipynb                  spark-warehouse
entry.py                        taxi_zone_lookup.csv
env                             yellow_tripdata_2024-10.parquet
result


In [79]:
df_zone = spark.read.option("header", "true").csv('taxi_zone_lookup.csv')


In [81]:
df_zone.registerTempTable('trips_zone')
df_zone.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [84]:
spark.sql(

    '''
    select zone, count(*) as ct from trips_data as d inner join trips_zone as z on z.LocationID = d.PULocationID group by zone order by ct asc 
    '''
).show()

+--------------------+---+
|                zone| ct|
+--------------------+---+
|Governor's Island...|  1|
|       Rikers Island|  2|
|       Arden Heights|  2|
|         Jamaica Bay|  3|
| Green-Wood Cemetery|  3|
|Charleston/Totten...|  4|
|   Rossville/Woodrow|  4|
|       West Brighton|  4|
|Eltingville/Annad...|  4|
|       Port Richmond|  4|
|         Great Kills|  6|
|        Crotona Park|  6|
|Heartland Village...|  7|
|     Mariners Harbor|  7|
|Saint George/New ...|  9|
|             Oakwood|  9|
|       Broad Channel| 10|
|New Dorp/Midland ...| 10|
|         Westerleigh| 12|
|     Pelham Bay Park| 12|
+--------------------+---+
only showing top 20 rows

